Parte A

Levantar cluster en AWS con Hadoop y Pyspark
Nombre: cluster_ + la mátricula (número de estudiante) más chica
Puntos a ajustar:

Red y seguridad da click en Grupos de seguridad de EC2 (firewall) * **Nodo principal** y * **Nodos principales y de tareas**, liga del grupo de seguridad * Da click en **reglas de entrada** (inbound rules):
Camiar puertos 22, 9443 y 18080 a My IP

Configuración del Perfil de la Instancia, cambiar por el nombre del bucket (s3)
"arn:aws:s3:::itam-analytics-MINOMBRE",       <--- AGREGAR
"arn:aws:s3:::itam-analytics-MINOMBRE/*",     <--- AGREGAR
                
"arn:aws:s3:::itam-analytics-MINOMBRE",        <--- AGREGAR
"arn:aws:s3:::itam-analytics-MINOMBRE/*",       <--- AGREGAR

Acceso a JupyterHub
JupyterHub:https://master-public-dns-name:9443/
u `jovyan`
p `jupyter`

NOTA: ajustar el nombre del bucket

Carga el CSV en Spark

In [4]:
# Revisión de estructura de datos antes de tener los headers
# Crear sesión de SPARK y Cargar los datos 

from pyspark.sql import SparkSession

# Inicia una sesión Spark.
spark = SparkSession.builder.appName("CSV to Parquet").getOrCreate()

# Lee el archivo CSV desde S3.
df = spark.read.csv("s3://qqa-completo/05-2023_01.csv", header=True, inferSchema=True)

# Muestra las primeras filas para asegurarte de que se ha cargado correctamente.
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+---------------------------------+--------------------+---------------------+-----------------+-------+----------+------+---------------------------+--------------------------------------+------------------------------------------------------------------------------------------+----------------+----------------+---------+-----------+
| ASISTENTES DE VOZ|ECHO SHOW 5. COLOR BLANCA O NEGRA|               ALEXA|APARATOS ELECTRONICOS|ELECTRODOMESTICOS|2499.00|2023-05-02|COPPEL|TIENDA DE ELECTRODOMÉSTICOS|COPPEL SUCURSAL ESPACIO AGUASCALIENTES|TECNOLOGICO 120, INT. CENTRO COMERCIAL ESPACIO AGUASCALIENTES, COL. OJOCALIENTE, CP. 20000|AGUASCALIENTES11|AGUASCALIENTES12|21.879490|-102.260011|
+------------------+---------------------------------+--------------------+---------------------+-----------------+-------+----------+------+---------------------------+--------------------------------------+--------------------------------------------------------------------------------

Carga el CSV en Spark
Adición de headers

In [22]:
# Crear sesión de SPARK, cargar los datos, insertar los nombres de las columnas

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("AddHeader").getOrCreate()

# Carga el archivo CSV sin encabezados.
df = spark.read.csv("s3://qqa-completo/05-2023_01.csv", header=False, inferSchema=True)

# Renombrar las columnas del DataFrame.
df_with_headers = df.toDF("producto", "presentacion", "marca", "categoria", "catalogo", "precio", "fecha_registro", "cadena_comercial", "giro", "nombre_comercial", "direccion", "estado", "municipio", "latitud", "longitud")

# Muestra las primeras filas para asegurarte de que se han cargado correctamente con los nuevos encabezados.
df_with_headers.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+--------------------+-----------------+-------+--------------+----------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+
|          producto|        presentacion|               marca|           categoria|        catalogom| precio|fecha_registro|cadena_comercial|                giro|    nombre_comercial|           direccion|        estado|     municipio|  latitud|   longitud|
+------------------+--------------------+--------------------+--------------------+-----------------+-------+--------------+----------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+
| ASISTENTES DE VOZ|ECHO SHOW 5. COLO...|               ALEXA|APARATOS ELECTRON...|ELECTRODOMESTICOS| 2499.0|    2023-05-02|          COPPEL|TIENDA DE ELECTRO...|COPPEL SUCURSAL E...|TECNOLOGICO 120, ...|AGUASCALIENTES|AGUASCALIE

Guarda el CSV como parquet en S3, particionalo por catalogo. (Utiliza todos los trucos que consideres).

In [9]:
# Guarda el CSV como parquet en S3, particionalo por catalogo
df_with_headers.write.partitionBy("catalogom").parquet("s3://qqa-completo/output/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Carga el parquet en Spark

In [10]:
# Carga del parquet en Spark
parquet_df = spark.read.parquet("s3://qqa-completo/output/")

# Muestra las primeras filas para verificar.
parquet_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+--------------------+--------------------+-------+--------------+----------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+-----------------+
|          producto|        presentacion|               marca|           categoria| precio|fecha_registro|cadena_comercial|                giro|    nombre_comercial|           direccion|        estado|     municipio|  latitud|   longitud|        catalogom|
+------------------+--------------------+--------------------+--------------------+-------+--------------+----------------+--------------------+--------------------+--------------------+--------------+--------------+---------+-----------+-----------------+
| ASISTENTES DE VOZ|ECHO SHOW 5. COLO...|               ALEXA|APARATOS ELECTRON...| 2499.0|    2023-05-02|          COPPEL|TIENDA DE ELECTRO...|COPPEL SUCURSAL E...|TECNOLOGICO 120, ...|AGUASCALIENTES|AGUASCALIENTES|21.879490|-10

¿Cuántos catálogos diferentes tenemos?

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, to_date, countDistinct

catalog_count_by_year = (parquet_df
    .withColumn("fecha_registro", to_date("fecha_registro", "yyyy-MM-dd"))  # Convierte la cadena de fecha en tipo de dato Date
    .groupBy(year("fecha_registro").alias("año"))  # Extrae el año y agrupa por este
    .agg(countDistinct("catalogom").alias("distinct_catalogs"))  # Cuenta los catálogos únicos
)

# Muestra el resultado.
catalog_count_by_year.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------------+
| a?o|distinct_catalogs|
+----+-----------------+
|2023|                7|
+----+-----------------+

¿Cuáles son los 20 catálogos con más observaciones? Guarda la salida de este query en tu bucket de S3, lo necesitaremos más adelante.

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Encuentra los 20 catálogos con más observaciones.
top_catalogs = parquet_df.groupBy("catalogom").count().orderBy(col("count").desc()).limit(20)

# Muestra el resultado.
top_catalogs.show()

# Guarda la salida en el bucket de S3.
top_catalogs.write.mode("overwrite").parquet("s3://qqa-completo/top_catalogs/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------+
|          catalogom| count|
+-------------------+------+
|            BASICOS|283314|
|       MEDICAMENTOS|112418|
|  ELECTRODOMESTICOS| 37306|
| FRUTAS Y LEGUMBRES| 32185|
|              PACIC| 26481|
|           MERCADOS| 11745|
|PESCADOS Y MARISCOS|  2282|
+-------------------+------+

¿Tenemos datos de todos los estados del país? De no ser así, ¿cuáles faltan?

In [16]:
from pyspark.sql import SparkSession

# Asumimos que 'df' es tu DataFrame y tiene una columna llamada 'state'.
# Lista todos los estados únicos presentes en el DataFrame.
unique_states = parquet_df.select("estado").distinct()

# Muestra los estados únicos.
unique_states.show(32)

# Opcional: Si quieres guardar esta lista en un archivo CSV en S3, puedes hacer lo siguiente:
unique_states.write.mode("overwrite").csv("s3://qqa-completo/unique_states/")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|              estado|
+--------------------+
|             YUCATÁN|
|            VERACRUZ|
|              OAXACA|
|              PUEBLA|
|        QUINTANA ROO|
|             TABASCO|
|             SINALOA|
|             CHIAPAS|
|              SONORA|
|             DURANGO|
|            TLAXCALA|
|COAHUILA DE ZARAGOZA|
|           CHIHUAHUA|
|     SAN LUIS POTOSÍ|
|          TAMAULIPAS|
| BAJA CALIFORNIA SUR|
|             JALISCO|
|          NUEVO LEÓN|
|    ESTADO DE MÉXICO|
|           ZACATECAS|
+--------------------+
only showing top 20 rows

In [21]:
# Asume que all_states es una lista de todos los estados posibles en tu país.
all_states = ["AGUASCALIENTES", "BAJA CALIFORNIA", "BAJA CALIFORNIA SUR", "CAMPECHE", "CHIAPAS","CHIHUAHUA","CIUDAD DE MEXICO",
"COAHUILA DE ZARAGOZA","COLIMA","DURANGO","ESTADO DE MEXICO","GUANAJUATO","GUERRERO","HIDALGO","JALISCO","MICHOACÁN","MORELOS","NAYARIT",
"NUEVO LEON","OAXACA","PUEBLA","QUERETARO","QUINTANA ROO","SAN LUIS POTOSI","SINALOA","SONORA","TABASCO","TAMAULIPAS","TLAXCALA",
"VERACRUZ","YUCATAN","ZACATECAS"]

# Crea un DataFrame para la lista completa de estados.
all_states_df = spark.createDataFrame([(state,) for state in all_states], ["estado"])

# Realiza un anti join para encontrar los estados que faltan.
missing_states = all_states_df.join(unique_states, "estado", "left_anti")

# Muestra los estados que faltan.
missing_states.show()

# Opcional: Guarda los estados que faltan en S3.
missing_states.write.mode("overwrite").csv("s3://qqa-completo/missing_states/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+
|          estado|
+----------------+
|CIUDAD DE M?XICO|
|          COLIMA|
|ESTADO DE M?XICO|
|       MICHOAC?N|
|         NAYARIT|
|      NUEVO LE?N|
|       QUER?TARO|
| SAN LUIS POTOS?|
|         YUCAT?N|
+----------------+

¿Cuántas observaciones tenemos por estado?

In [28]:
from pyspark.sql.functions import col

# Agrupa por estado y cuenta las observaciones
observations_per_state = parquet_df.groupBy("estado").count().orderBy(col("count").desc())

# Muestra los resultados
observations_per_state.show(32)

# Si necesitas guardar estos resultados, puedes hacerlo en un archivo CSV en tu bucket de S3:
observations_per_state.write.mode("overwrite").csv("s3://qqa-completo/observations_per_state/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------+
|             estado| count|
+-------------------+------+
|   CIUDAD DE MÉXICO|117779|
|   ESTADO DE MÉXICO| 80309|
|            JALISCO| 24028|
|         GUANAJUATO| 23527|
|         NUEVO LEÓN| 17754|
|            TABASCO| 17120|
|             PUEBLA| 16857|
|          QUERÉTARO| 16517|
|          CHIHUAHUA| 15750|
|            YUCATÁN| 14231|
|          ZACATECAS| 13775|
|       QUINTANA ROO| 12486|
|           CAMPECHE| 11650|
|BAJA CALIFORNIA SUR| 10338|
|     AGUASCALIENTES|  9957|
|            DURANGO|  9878|
|           TLAXCALA|  9766|
|MICHOACÁN DE OCAMPO|  9703|
|             SONORA|  9055|
|             OAXACA|  8813|
+-------------------+------+
only showing top 20 rows

De cada estado obten: el número de catalogos diferentes por año, ¿ha aumentado el número de catálogos con el tiempo?

In [38]:
from pyspark.sql.functions import col, countDistinct, lag, year, to_date
from pyspark.sql.window import Window

# Define una ventana particionada por estado y ordenada por año para análisis temporal.
windowSpec = Window.partitionBy("estado").orderBy("year")

# Preparar el DataFrame y calcular el número de catálogos distintos por estado y año, y el cambio año a año.
catalogs_per_state_year = (
    parquet_df
    # Asegurarse de que 'fecha_registro' está en formato de fecha correcto.
    .withColumn("fecha_registro", to_date("fecha_registro", "yyyy-MM-dd"))
    # Extraer el año y agrupar por este y por estado.
    .groupBy("estado", year("fecha_registro").alias("year"))
    .agg(countDistinct("catalogom").alias("num_catalogs"))
    # Calcular el número de catálogos del año anterior y el cambio respecto al año anterior.
    .withColumn("prev_year_catalogs", lag("num_catalogs", 1).over(windowSpec))
    .withColumn("change", col("num_catalogs") - col("prev_year_catalogs"))
    .select("estado", "year", "num_catalogs", "change")
    .orderBy("estado", "year")
)

# Muestra los resultados
catalogs_per_state_year.show()

# Si necesitas guardar estos resultados, puedes hacerlo en un archivo CSV en tu bucket de S3:
catalogs_per_state_year.write.mode("overwrite").csv("s3://qqa-completo/catalogs_per_state_year/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+------------+------+
|              estado|year|num_catalogs|change|
+--------------------+----+------------+------+
|      AGUASCALIENTES|2023|           7|  NULL|
|     BAJA CALIFORNIA|2023|           7|  NULL|
| BAJA CALIFORNIA SUR|2023|           7|  NULL|
|            CAMPECHE|2023|           7|  NULL|
|             CHIAPAS|2023|           7|  NULL|
|           CHIHUAHUA|2023|           7|  NULL|
|    CIUDAD DE MÉXICO|2023|           7|  NULL|
|COAHUILA DE ZARAGOZA|2023|           7|  NULL|
|             DURANGO|2023|           7|  NULL|
|    ESTADO DE MÉXICO|2023|           7|  NULL|
|          GUANAJUATO|2023|           7|  NULL|
|            GUERRERO|2023|           3|  NULL|
|             HIDALGO|2023|           7|  NULL|
|             JALISCO|2023|           7|  NULL|
| MICHOACÁN DE OCAMPO|2023|           6|  NULL|
|             MORELOS|2023|           7|  NULL|
|          NUEVO LEÓN|2023|           7|  NULL|
|              OAXACA|2023|           5|